In [1]:
from bs4 import BeautifulSoup
import requests
import re
import random
import datetime
from tkinter import *
from tkinter import ttk
#from datetime import datetime

url = 'https://reatch.ch/veranstaltungen'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

links = soup.find_all('a')
hrefs = []
for link in links:
    href = link.get('href')
    if href and 'veranstaltungen' in href:
        hrefs.append(href)

# Remove duplicates by converting the list to a set and then back to a list
hrefs = list(set(hrefs))

# Filter out URLs that do not contain "https://reatch.ch/veranstaltungen/"
hrefs = [href for href in hrefs if "https://reatch.ch/veranstaltungen/" in href]

# Print the contents of the list
for href in hrefs:
    print(href)


https://reatch.ch/veranstaltungen/franxini-boot-camp-sprint-2
https://reatch.ch/veranstaltungen/iceland-meltdown
https://reatch.ch/veranstaltungen/wohnen-im-alter
https://reatch.ch/veranstaltungen/pizza-philosophy-and-science-lebenswerte-staedte-dank-regenwasser-stadtplanung-in-zeiten-des-klimawandels
https://reatch.ch/veranstaltungen/brainf-ck-improvisationstheater-trifft-wissenschaft-18
https://reatch.ch/veranstaltungen/brainf-ck-improvisationstheater-trifft-wissenschaft-17
https://reatch.ch/veranstaltungen/reatching-into-the-rabbit-hole-vision-2080-zur-beziehung-mensch-natur
https://reatch.ch/veranstaltungen/brainf-ck-improvisationstheater-trifft-wissenschaft-19
https://reatch.ch/veranstaltungen/pizza-philosophy-and-science-psychedelic-research-trends
https://reatch.ch/veranstaltungen/finanzplatz-schweiz


In [2]:
# Assuming hrefs is your list of URLs
def scrape_all_links(hrefs):
    for index, url in enumerate(hrefs):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Create a global variable with the specified name and assign the URL to it
        globals()[f'MESSAGE{index+1}'] = url

        # Create a counter for the text bits
        counter = 1

        # Find all div elements with class "entry-header"
        divs = soup.find_all('div', class_='entry-header')
        for div in divs:
            # Find h1 element within each div
            header = div.find('h1')
            if header:
                # Create a global variable with the specified name and assign the header text to it
                globals()[f'MESSAGE{index+1}_HEADER'] = header.get_text()

            # Find all p elements within each div
            ps = div.find_all('p')
            for p in ps:
                # Create a global variable with the specified name and assign the paragraph text to it
                text = p.get_text()
                
                # If this is the second text bit, remove paragraphs by replacing newline characters with spaces
                if counter == 2:
                    text = text.replace('\n', ' ')
        
                
                globals()[f'MESSAGE{index+1}_TEXT{counter}'] = text
                counter += 1

# Call the function with your list of URLs
scrape_all_links(hrefs)


In [3]:
# Get a list of all global variable names
global_vars = list(globals().keys())

# Filter the list to only include variables that start with "LINK"
link_vars = [var for var in global_vars if var.startswith('MESSAGE')]

In [4]:
# Get a list of all global variable names
global_vars = list(globals().keys())

# Filter the list to only include variables that end with '_TEXT2'
text_vars = [var for var in global_vars if var.endswith('_TEXT2')]

# Iterate over each variable name
for var in text_vars:
    # Get the value of the variable
    value = globals()[var]
    
    # Add a space before the hyphen if it is immediately preceded by a character
    globals()[var] = re.sub(r'(\w)-', r'\1 -', value)


In [5]:
# Assuming hrefs is your list of URLs
def modify_header_text(hrefs):
    for index, url in enumerate(hrefs):
        link_num = index + 1
        header_var = f'MESSAGE{link_num}_HEADER'
        if header_var in globals():
            header_text = globals()[header_var]
            # Check if the header text already contains an asterisk
            if '*' in header_text:
                # Enclose the header text in quotation marks, then add asterisks at the beginning and end
                modified_text = f'*"{header_text}"*'
            else:
                # Add an asterisk at the beginning and end of the header text
                modified_text = f'*{header_text}*'
            # Update the global variable with the modified text
            globals()[header_var] = modified_text

modify_header_text(hrefs)


In [17]:
import random
import re
from datetime import datetime

def combine_link_text(hrefs):
    templates = [
        'Salü zeme, bald isch wieder en coole Event vellecht hesch au Ziit zum cho! Alli Infos sind unde :)',
        'Hallo, vellecht findsch du die Verastaltig spannend! Gnaueri Infos findsch uf de Website 😁', 
        'Hoi zeme, nöchst Wuche findet en spannende Event statt, chum doch au verbi! Meh Infos findsch uf de Website :)', 
        'Sali mitenand, bald isch es sowit für de cooli Event. Bisch au debi? Gnaueres findsch unde 😁', 
        'Hey zeme, nöchstens findet wirds wieder en interessanti Verastaltig gäh, wär schön wänn du au chunsch! Alli Infos findsch uf de Website!',
    ]

    events = []

    for index, url in enumerate(hrefs):
        event_info = {'text': ''}

        link_num = index + 1
        header_var = f'MESSAGE{link_num}_HEADER'
        if header_var in globals():
            header_text = globals()[header_var]
            event_info['text'] = header_text + '\n\n'

        counter = 1
        while True:
            text_var = f'MESSAGE{link_num}_TEXT{counter}'
            if text_var in globals():
                text = globals()[text_var]
                event_info['text'] += text + '\n'
                counter += 1
            else:
                break

        # Extract the date from the description using regular expressions
        date_match = re.search(r'\d{2}\.\d{2}\.\d{4}', event_info['text'])
        if date_match:
            date_str = date_match.group()
            event_info['date'] = datetime.strptime(date_str, '%d.%m.%Y')

        event_info['url'] = url
        events.append(event_info)

    # Sort events by date
    events.sort(key=lambda x: x['date'])

    for event in events:
        template = random.choice(templates)
        event_text_lines = event['text'].strip().split('\n')

        # Remove irrelevant lines and ensure no unnecessary spaces before the date
        relevant_lines = []
        for line in event_text_lines:
            if line.strip() and not re.match(r'^(max\.|Details|Free)', line.strip()):
                relevant_lines.append(line.strip())

        # Reconstruct the text
        event['text'] = '\n'.join(relevant_lines)
        combined_text_with_template = template + '\n\n' + event['text'] + '\nURL: ' + event['url']

        print('\nMESSAGE:\n')
        print(combined_text_with_template + '\n')

# Example usage
combine_link_text(hrefs)



MESSAGE:

Sali mitenand, bald isch es sowit für de cooli Event. Bisch au debi? Gnaueres findsch unde 😁

*Reatching into the Rabbit Hole: Brieffreundschaften*
Bist du bereit für eine Reise in eine neue Welt in der wir Menschen voneinander und unserer Natur ein ganz anderes Verständnis entwickelt haben? In unserer Vision 2080, gestalten wir gemeinsam und untereinander unsere Beziehung zur Natur.
Wie? Anhand von Brieffreundschaften!
Anmeldeschluss: 15. Januar 2024
17.02.2024
URL: https://reatch.ch/veranstaltungen/reatching-into-the-rabbit-hole-vision-2080-zur-beziehung-mensch-natur


MESSAGE:

Salü zeme, bald isch wieder en coole Event vellecht hesch au Ziit zum cho! Alli Infos sind unde :)

*Pizza, Philosophy and Science: Lebenswerte Städte dank Regenwasser – Stadtplanung in Zeiten des Klimawandels*
Anstatt weisse Weihnachten zu geniessen, kämpften die Menschen in Deutschland am Jahresende mit Starkregen und Hochwasser. Solche Extremwetterereignisse werden uns in Zukunft immer häufiger 